<a href="https://colab.research.google.com/github/iljeong/DATATHON_5/blob/main/%EB%A9%8B%EC%82%AC_%EB%8D%B0%EC%9D%B4%ED%84%B0%ED%86%A4_5%EC%A1%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gcar/Socar 리뷰 크롤링
- Google Play Store 리뷰 데이터 수집
- Apple App Store 리뷰 데이터 수집

In [ ]:
import time
import requests
import pandas as pd
from google_play_scraper import Sort, reviews

# ==============================
# Google Play 리뷰 수집
# ==============================
def crawl_google_reviews(app_id, provider_name, lang='ko', country='kr', max_count=3000):
    """
    Google Play 리뷰 크롤링
    - app_id: Google Play 앱 ID (예: 'com.greencar')
    - provider_name: 'Gcar' 또는 'Socar'
    - max_count: 최대 수집 개수
    """
    all_reviews = []
    next_token = None
    total = 0

    print(f"\n📱 [Google Play] {provider_name} 리뷰 수집 시작...")

    while True:
        rv_list, next_token = reviews(
            app_id,
            lang=lang,
            country=country,
            sort=Sort.NEWEST,
            continuation_token=next_token
        )

        if not rv_list:
            break

        for r in rv_list:
            rating = r.get("score")
            review_text = r.get("content")
            author = r.get("userName")
            version = r.get("reviewCreatedVersion")
            at = r.get("at")

            if at is not None:
                review_date = at.strftime("%Y-%m-%d %H:%M")
            else:
                review_date = None

            all_reviews.append({
                "provider": provider_name,
                "store": "Google Play Store",
                "rating": rating,
                "review_text": review_text,
                "review_date": review_date,
                "author": author,
                "version": version,
            })

        total += len(rv_list)
        print(f"  → 누적 {total}개 수집")

        if not next_token or total >= max_count:
            break

        time.sleep(1.5)

    df = pd.DataFrame(all_reviews)
    print(f"[Google Play] {provider_name} 최종 수집 개수: {len(df)}")
    return df


# ==============================
# Apple App Store 리뷰 수집
# ==============================
def crawl_appstore_reviews(app_id, provider_name, country='kr', max_page=20, sort='recent'):
    """
    App Store 리뷰 크롤링 (RSS)
    - app_id: App Store 앱 ID (숫자 문자열)
    - provider_name: 'Gcar' 또는 'Socar'
    - country: 'kr' (한국)
    - sort: 'recent'(최신순) / 'helpful'(도움순) 등
    """
    print(f"\n🍎 [App Store] {provider_name} 리뷰 수집 시작... (정렬: {sort})")

    base_url = (
        f"https://itunes.apple.com/{country}/rss/customerreviews/"
        f"page={{page}}/id={app_id}/sortby={sort}/json"
    )

    all_reviews = []
    for page in range(1, max_page + 1):
        url = base_url.format(page=page)
        resp = requests.get(url)
        if resp.status_code != 200:
            print(f"  ⚠️ page {page}: HTTP {resp.status_code}, 중단")
            break

        data = resp.json()
        entries = data.get("feed", {}).get("entry", [])

        if len(entries) <= 1:
            print("  ⚠️ 더 이상 리뷰 없음, 중단")
            break

        for entry in entries[1:]:
            rating = int(entry.get("im:rating", {}).get("label", 0))
            review_text = entry.get("content", {}).get("label")
            author = entry.get("author", {}).get("name", {}).get("label")
            version = entry.get("im:version", {}).get("label")
            updated = entry.get("updated", {}).get("label")

            if updated:
                try:
                    dt = pd.to_datetime(updated)
                    review_date = dt.strftime("%Y-%m-%d %H:%M")
                except Exception:
                    review_date = updated
            else:
                review_date = None

            all_reviews.append({
                "provider": provider_name,
                "store": "App Store",
                "rating": rating,
                "review_text": review_text,
                "review_date": review_date,
                "author": author,
                "version": version,
            })

        print(f"  → page {page} 처리, 누적 {len(all_reviews)}개")
        time.sleep(1)

    df = pd.DataFrame(all_reviews)
    print(f"[App Store] {provider_name} 최종 수집 개수: {len(df)}")
    return df


# ==============================
# 실제 실행 부분
# ==============================
if __name__ == "__main__":

    google_apps = {
        "Gcar": "com.greencar",
        "Socar": "kr.co.socar",
    }

    appstore_apps = {
        "Gcar": "834786891",
        "Socar": "869016361",
    }

    # ---------- Google Play ----------
    google_frames = []
    for provider, app_id in google_apps.items():
        df_g = crawl_google_reviews(
            app_id=app_id,
            provider_name=provider,
            max_count=20000
        )
        google_frames.append(df_g)

    # ---------- App Store ----------
    appstore_frames = []
    for provider, app_id in appstore_apps.items():
        df_a = crawl_appstore_reviews(
            app_id=app_id,
            provider_name=provider,
            sort="recent",
            max_page=50
        )
        appstore_frames.append(df_a)

    # ==============================
    # 데이터 병합 + ID 부여 + 컬럼 정리
    # ==============================
    all_df = pd.concat(google_frames + appstore_frames, ignore_index=True)

    all_df["review_date"] = all_df["review_date"].astype(str)

    all_df.insert(
        0,
        "ID",
        ["review_" + str(i).zfill(2) for i in range(1, len(all_df) + 1)]
    )

    # 감정분석 / 카테고리 컬럼 형식 만들어두기
    all_df["Sentiment_label"] = None
    all_df["Sentiment_score"] = None
    all_df["Category"] = None

    # 최종 컬럼 순서
    final_cols = [
        "ID",
        "provider",
        "store",
        "rating",
        "review_text",
        "review_date",
        "author",
        "version",
        "Sentiment_label",
        "Sentiment_score",
        "Category",
    ]
    all_df = all_df[final_cols]

    # ==============================
    # 저장
    # ==============================
    output_path = "reviews_sentiment_all.csv"
    all_df.to_csv(output_path, index=False, encoding="utf-8-sig")
    print(f"\n🎉 완료! → {output_path} 로 저장됨")

# 카카오 맵 API 호출
- 차고지 데이터 수집

In [ ]:
# ==========================================================
# 전국 쏘카존 / 지카존 완전 수집 (누락 최소화, 중복 제거 없음)
# ==========================================================

REST_API_KEY = "d47532baa79392c04b6c3cb99713a66f"

# 주요 키워드 (대소문자·표기 변형 포함)
KEYWORDS = ["G car zone"]

# Kakao Local API 한계
RADIUS_M = 20000      # 최대 반경 20km
GRID_STEP_KM = 12     # 12km 간격 격자 (촘촘하게 전국 커버)

# 대한민국 대략 경계
MIN_LAT, MAX_LAT = 33.10, 38.70
MIN_LNG, MAX_LNG = 124.60, 131.95
# ==========================================================

import os, math, time, json, requests, pandas as pd, pathlib
from tqdm import tqdm

# 인증 헤더 세팅
os.environ["KAKAO_REST_KEY"] = REST_API_KEY
HEADERS = {"Authorization": f"KakaoAK {os.environ['KAKAO_REST_KEY']}"}

# ======== 도우미 함수 ========
def km_to_deg_lat(km): return km / 111.0
def km_to_deg_lng(km, lat): return km / (111.0 * math.cos(math.radians(lat)))

def build_grid(min_lat, max_lat, min_lng, max_lng, step_km):
    points = []
    dlat = km_to_deg_lat(step_km)
    lat = min_lat
    while lat <= max_lat:
        dlng = km_to_deg_lng(step_km, lat)
        lng = min_lng
        while lng <= max_lng:
            points.append((round(lat, 6), round(lng, 6)))
            lng += dlng
        lat += dlat
    return points

def search_keyword_all(query, lat, lng, radius=RADIUS_M, size=15, max_pages=45, pause=0.12):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    out = []
    for page in range(1, max_pages + 1):
        params = {"query": query, "y": lat, "x": lng, "radius": radius, "size": size, "page": page}
        r = requests.get(url, params=params, headers=HEADERS, timeout=15)
        r.raise_for_status()
        data = r.json()
        out.extend(data.get("documents", []))
        if data.get("meta", {}).get("is_end"): break
        time.sleep(pause)
    return out

# ======== 격자 생성 ========
grid_points = build_grid(MIN_LAT, MAX_LAT, MIN_LNG, MAX_LNG, GRID_STEP_KM)
print(f"격자 중심점 개수: {len(grid_points)}  (전국 커버 약 {len(grid_points)*RADIUS_M/1000/GRID_STEP_KM:.1f}%)")

# ======== 수집 시작 ========
rows = []
for (lat, lng) in tqdm(grid_points):
    for kw in KEYWORDS:
        try:
            docs = search_keyword_all(kw, lat, lng)
        except Exception as e:
            continue
        for d in docs:
            rows.append({
                "query": kw,
                "grid_lat": lat,
                "grid_lng": lng,
                "name": d.get("place_name"),
                "category": d.get("category_name"),
                "address": d.get("address_name"),
                "road_address": d.get("road_address_name"),
                "lat": float(d["y"]) if d.get("y") else None,
                "lng": float(d["x"]) if d.get("x") else None,
                "distance_m": int(d["distance"]) if d.get("distance") else None,
                "phone": d.get("phone"),
                "url": d.get("place_url")
            })
    time.sleep(0.2)

# ======== 결과 저장 ========
df = pd.DataFrame(rows)
print(f"\n✅ 총 {len(df)}건 수집 완료 (중복 포함)")

ROOT = pathlib.Path("kakao_output_nationwide")
ROOT.mkdir(exist_ok=True)
csv_path  = ROOT / "kakao_places_socar_gcar_nationwide.csv"
json_path = ROOT / "kakao_places_socar_gcar_nationwide.json"

df.to_csv(csv_path, index=False, encoding="utf-8-sig")
df.to_json(json_path, orient="records", force_ascii=False, indent=2)
print(f"\n저장 완료 ✅\nCSV : {csv_path}\nJSON: {json_path}")

# 지역 요약 확인
if not df.empty:
    df["region"] = df["address"].fillna("").apply(lambda s: s.split()[0] if s else "")
    print("\n📊 지역별 분포 Top 15")
    print(df["region"].value_counts().head(15))

# 리뷰 데이터 전처리
- 다른 모빌리티 서비스(일레클/딜카/패스카/카카오/모빌리티) 언급 리뷰 제외
- 이모지/특수문자 제거
- 공백 정리
- Okt 명사 추출(안 되면 정규식 토큰화)
- 불용어 제거

In [ ]:
# Java 설치
!apt-get update
!apt-get install -y openjdk-11-jdk

# 환경 변수 설정
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["PATH"] += os.pathsep + os.path.join(os.environ["JAVA_HOME"], "bin")

# konlpy 설치
!pip install konlpy

In [ ]:
# 1. 기본 셋업
import os
import re
import random

import numpy as np
import pandas as pd


# 재현성(랜덤 고정)
SEED = 42
random.seed(SEED)
np.random.seed(SEED)


# 입력 / 출력 경로 설정
INPUT_PATH = "/content/CarSharing_Reviews_20251029_id.csv"
OUTPUT_PATH = "/content/reviews_preprocessed.csv"


# 2. 토크나이저: Okt → 안되면 폴백
def get_tokenizer():
    try:
        from konlpy.tag import Okt
        okt = Okt()

        def tokenize_ko(text: str):
            text = str(text)
            morphs = []
            for w, pos in okt.pos(text, norm=True, stem=True):
                # 명사만 사용 (필요하면 형용사/동사도 추가 가능)
                if pos == "Noun":
                    # 한 글자 짜리 명사는 노이즈가 많아서 제거
                    if len(w) > 1:
                        morphs.append(w)
            return morphs

        print("[INFO] 토크나이저: Okt 사용")
        return tokenize_ko

    except Exception as e:
        # JVM 설치 안 되어 있거나 konlpy 문제 있을 때
        print(f"[WARN] Okt 사용 불가, fallback 토크나이저 사용: {e}")

        def simple_tokenize_ko(text: str):
            text = re.sub(r"[^가-힣A-Za-z0-9\s]", " ", str(text))
            # 한글 2글자 이상만 남기기
            toks = re.findall(r"[가-힣]{2,}", text)
            return toks

        return simple_tokenize_ko


tokenize_ko = get_tokenizer()


# 3. 데이터 로드
df = pd.read_csv(INPUT_PATH)
print(f"[INFO] 원본 리뷰 수: {len(df):,}건")

# 필수 컬럼 체크 (review_text 기준)
assert "review_text" in df.columns, "CSV에 'review_text' 컬럼이 필요합니다."

# 날짜 컬럼 자동 탐색
date_col = None
for c in df.columns:
    if "date" in c.lower():
        date_col = c
        break
print(f"[INFO] 날짜 컬럼 자동 탐지 결과: {date_col}")


# 4. 타 서비스 언급 제거
#    (일레클, 딜카, 패스카, 카카오, 모빌리티 등)
exclude_keywords = ['일레클', '딜카', '패스카', '카카오', '모빌리티']

# 조사/어미 붙은 경우(일레클은, 카카오는, 모빌리티에서 등)까지 같이 제거
pattern = r'(' + '|'.join(map(re.escape, exclude_keywords)) + r')(은|는|이|가|을|를|에|에서|으로|와|과|의|도)?'

before = len(df)
df = df[~df['review_text'].astype(str).str.contains(pattern, case=False, na=False)].copy()
df.reset_index(drop=True, inplace=True)
print(f"[INFO] 타 서비스 언급 제거: {before - len(df)}건 제거, 잔여 {len(df):,}건")


# 5. 텍스트 정리 (특수문자 제거 + 공백 정리)
def clean_text(text: str) -> str:
    text = str(text)
    # 이모지/특수문자 제거(한글,영문,숫자,공백만 남기기)
    text = re.sub(r"[^가-힣A-Za-z0-9\s]", " ", text)
    # 연속 공백 → 한 칸
    text = re.sub(r"\s+", " ", text)
    return text.strip()

df["clean_text"] = df["review_text"].astype(str).apply(clean_text)


# 6. 불용어 정의
STOPWORDS = set([
    "하다","되다","이다","있다","없다","같다","보다","주다","받다","되",
    "좋다","나쁘다","자다","됨","되고","해서","하면","하는","했다","했던",
    "같은","든","다시","예","아이고","아휴","하","허","후",
    "의","가","이","은","는","을","를","에","로","도","만","와","과","및",
    "그리고","마다","에서","으로","에게","까지","때문","때문에","지만","거나",
    "때","거","수","정말","너무","매우","진짜","완전","계속","그냥","좀","잘",
    "더","덜","막","또","등","라","데","요","니다","그린","이용","사용","서비스",
    "차량","운전","오늘","어제","내일","이번","저번","다른","없고","없다고",
    "없음","없이","없는","근데","그럼","전에","그렇게","이게","이런","이렇게",
    "처음","바로","지금","결국","무슨","절대","많이","전혀","아니","아니고","아예",
    "안되고","안되서","안된다고","안됨","안받고","못하고","해도","했더니","대한",
    "저는","제가","내가","회사","고객이","합니다","문이","차가","차를","차량이",
    "차량을","그린카","그린카는","쏘카","쏘카는","쏘카를","사용하다","그렇게",
    "갑자기","있어서","이용하다","없어서","거예요","거네요","거같아요","거같음",
    "아","어","음","헐","휴","우와","에휴","진짜로","와","흠","아니요","네","응",
    "그래서","그러니까","그런데","그럼에도","그러면","아니면","때문인지",
    "때문인지도","왜냐면","그렇지만","또한","그리고나서","그래도","그런가",
    "그랬더니","그렇다보니","하게","하면서도","하려고","하려니",
    "까지는","처럼","정도","대로","뿐","따라","마다","부터","만큼","하면서",
    "그나마","조금","살짝","약간","되게","엄청","너무나","굉장히","항상",
    "계속해서","매번","대체로","거의","아주","완전히","도대체",
    "좀더","빨리","늦게","처리","제발","문의","요청","답변","불편",
    "문의했는데","문의드려요","개선","필요","해결","조치","문제","상황",
    "도요","으로는","에는","에게는","라고","이라고","네요","습니다","는데",
    "는데요","해서요","같아요","해요","하네요","네요.","같음","거에요"
])


# 7. 토큰화 + 불용어 제거
def tokenize_and_filter(text: str):
    # 1) 토큰화
    toks = tokenize_ko(text)
    # 2) 불용어 제거 + 길이 필터
    toks = [w for w in toks if (w not in STOPWORDS and len(w) > 1)]
    return toks

df["tokens"] = df["clean_text"].apply(tokenize_and_filter)

# 토큰을 공백 기준 문자열로도 만들어두면 TF-IDF, 워드클라우드 등에 편함
df["tokens_str"] = df["tokens"].apply(lambda xs: " ".join(xs))

print("[INFO] 전처리 예시 5건:")
print(df[["review_text", "clean_text", "tokens"]].head().to_string(index=False))


# 8. 날짜 컬럼 포맷 정리
if date_col:
    # 날짜를 datetime으로 변환 후, YYYY-MM-DD 형태 문자열로 저장
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce").dt.date.astype(str)


# 9. 전처리 결과 저장
df.to_csv(OUTPUT_PATH, index=False, encoding="utf-8-sig")
print(f"전처리 완료! → {OUTPUT_PATH} 로 저장됨")

# 긍/부정 모델링
- Sentiment_label : "긍정', "부정"
- Sentiment_score: 귿정확률 (0.0 ~ 1.0)

In [ ]:
!pip install transformers accelerate

# 1. 기본 셋업
import csv
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm.auto import tqdm


# 전처리 완료된 리뷰 파일
INPUT_PATH = "/content/reviews_preprocessed.csv"
OUTDIR = "/content/sentiment_out"
os.makedirs(OUTDIR, exist_ok=True)
OUTPUT_PATH = os.path.join(OUTDIR, "reviews_with_sentiment.csv")


# 2. 데이터 로드
df = pd.read_csv(INPUT_PATH)
print(f"[INFO] 입력 데이터: {len(df):,}건")

assert "review_text" in df.columns, "CSV에 'review_text' 컬럼이 필요합니다."

# (선택) 날짜 컬럼 자동 탐지
date_col = None
for c in df.columns:
    if "date" in c.lower():
        date_col = c
        break
print(f"[INFO] 날짜 컬럼: {date_col}")


# 3. KoELECTRA 감정분석 모델 로드
MODEL_NAME = "jaehyeong/koelectra-base-v3-generalized-sentiment-analysis"

print(f"[INFO] 모델 로드: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
print(f"[INFO] device = {device}")

# 라벨 인덱스 확인 (모델마다 0/1/2의 의미가 다를 수 있어서 안전하게 처리)
id2label = model.config.id2label
print("[INFO] id2label:", id2label)

# 긍정/중립/부정 인덱스 찾기
pos_idx = neu_idx = neg_idx = None
for i, lab in id2label.items():
    l = lab.lower()
    if "pos" in l or "긍정" in l:
        pos_idx = i
    elif "neg" in l or "부정" in l:
        neg_idx = i

# 혹시 못 찾으면 기본값 세팅
if pos_idx is None:
    pos_idx = 1
if neg_idx is None:
    neg_idx = 0

print(f"[INFO] label index - neg:{neg_idx}, neu:{neu_idx}, pos:{pos_idx}")

def map_label(idx: int) -> str:
    raw = id2label.get(idx, "").lower()
    if "pos" in raw or "긍정" in raw:
        return "긍정"
    if "neg" in raw or "부정" in raw:
        return "부정"
    # 혹시 애매하면 일단 부정/긍정으로 나눔
    return "긍정" if idx == pos_idx else "부정"


# 4. 배치 추론 함수 정의
def predict_batch(texts, batch_size=64, max_len=128):
    labels_kr = []
    pos_scores = []

    for i in tqdm(range(0, len(texts), batch_size)):
        batch = list(map(str, texts[i:i+batch_size]))

        # 토크나이즈
        inputs = tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_len,
        ).to(device)

        with torch.no_grad():
            logits = model(**inputs).logits
            probs = torch.softmax(logits, dim=1).cpu().numpy()

        # 각 샘플별로 확률과 라벨 뽑기
        for p in probs:
            # 원 모델의 argmax 라벨
            label_idx = int(p.argmax())
            labels_kr.append(map_label(label_idx))
            # 긍정 확률 (pos_idx 기준)
            pos_scores.append(float(p[pos_idx]))

    return labels_kr, pos_scores


# 5. 실제 감정분석 수행
texts = df["review_text"].fillna("").tolist()
print("[INFO] 감정분석 시작...")
sent_labels, sent_scores = predict_batch(texts, batch_size=96, max_len=128)

df["Sentiment_label"] = sent_labels       # "긍정/부정"
df["Sentiment_score"] = sent_scores       # 긍정 확률 (0~1 float)

print("[INFO] 감정분석 결과 예시 5건:")
print(df[["review_text", "Sentiment_label", "Sentiment_score"]].head().to_string(index=False))


# 6. 텍스트 정리 (CSV용 안전 처리)
def sanitize_text(s):
    s = "" if pd.isna(s) else str(s)
    return (
        s.replace("\r\n", " ")
         .replace("\n", " ")
         .replace("\t", " ")
         .replace("\x00", "")
    )

df["review_text"] = df["review_text"].map(sanitize_text)

# 날짜 컬럼이 있다면 포맷 통일
if date_col:
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce").dt.date.astype(str)


# 7. 저장할 컬럼만 골라서 CSV로 저장
save_cols = []

for col in ["ID", "provider", "store", "rating", date_col, "review_text",
            "Sentiment_label", "Sentiment_score"]:
    if col and col in df.columns:
        save_cols.append(col)

safe = df[save_cols].copy()

safe.to_csv(
    OUTPUT_PATH,
    index=False,
    encoding="utf-8-sig",
    quoting=csv.QUOTE_MINIMAL,
    lineterminator="\n"
)
print(f"감정분석 완료! → {OUTPUT_PATH} 로 저장됨")

# XAI

1. SHAP 셋업 (masker + 함수)

In [ ]:
import shap

masker = shap.maskers.Text(tokenizer)

def f_shap(texts, max_len=128):
    texts = list(map(str, texts))
    inputs = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_len
    ).to(device)

    with torch.no_grad():
        logits = model(**inputs).logits
        probs = torch.softmax(logits, dim=1).cpu().numpy()

    return probs[:, pos_idx]

2. SHAP Explainer 생성

In [ ]:
explainer = shap.Explainer(
    f_shap,
    masker,
    algorithm="partition"
)

3. 리뷰 샘플 선정 & SHAP 값 계산

In [ ]:
neg_samples = df[df["Sentiment_label"]=="부정"]["review_text"].head(2).tolist()
pos_samples = df[df["Sentiment_label"]=="긍정"]["review_text"].head(2).tolist()

sample_texts = neg_samples + pos_samples

print("샘플 개수:", len(sample_texts))

shap_values = explainer(
    sample_texts,
    max_evals=500,
    batch_size=8
)

4. 개별 리뷰별 단어 중요도 시각화

In [ ]:
for i, text in enumerate(sample_texts):
    score = float(f_shap([text])[0])
    print(f"\n[샘플 {i+1}] 긍정확률 = {score:.3f}")
    shap.plots.text(shap_values[i])

5. 전역(Global) 단어 중요도 계산
- 80개 리뷰 샘플링 후 SHAP 계산

In [ ]:
texts_80 = df["review_text"].dropna().sample(80, random_state=42).tolist()
shap_values_80 = explainer(texts_80)

- 긍정/부정 클래스별 SHAP 분해

In [ ]:
sv = shap_values_80[:,:,0]

- 전체 토큰별 평균 영향력 계산

In [ ]:
global_importance = sv.abs.mean(0)

- 상위 20개 긍정/부정 단어 추출

In [ ]:
tokens = sv.data

values = global_importance.values

# TOP 20 긍정(긍정 영향 ↑)
idx_top = np.argsort(-values)[:20]
top_pos_words = [(tokens[i], float(values[i])) for i in idx_top]

# TOP 20 부정(긍정 확률을 깎는 단어)
idx_neg = np.argsort(values)[:20]
top_neg_words = [(tokens[i], float(values[i])) for i in idx_neg]

print("긍정을 밀어주는 단어 TOP 20")
for w, v in top_pos_words:
    print(f"{w:>10s}  +{v:.4f}")

print("부정을 밀어주는 단어 TOP 20 (긍정을 깎는 단어)")
for w, v in top_neg_words:
    print(f"{w:>10s}  -{v:.4f}")

6. 전역 SHAP 바 차트 시각화

In [ ]:
import matplotlib.pyplot as plt

def plot_top_words(top_items, title):
    tokens_ = [t for t, v in top_items]
    vals_   = [v for t, v in top_items]

    plt.figure(figsize=(7,5))
    y = np.arange(len(tokens_))
    plt.barh(y, vals_)
    plt.yticks(y, tokens_)
    plt.gca().invert_yaxis()
    plt.title(title)
    plt.tight_layout()
    plt.show()

plot_top_words(top_pos_words, "긍정 영향 TOP 20 단어")
plot_top_words(top_neg_words, "부정 영향 TOP 20 단어")

# 리뷰데이터 평점 분포 & 검증
- 평균 평점 계산
- 파이 차트로 비교
- 정규성 검정
- 등분산성 검정
- Mann–Whitney U 검정

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats

# 1. 브랜드별 평균 평점 계산
mean_scores = df.groupby("provider")["rating"].mean()
print("브랜드별 평균 평점:")
print(mean_scores)
print("=" * 50)


# 2. 평점 분포 시각화 (파이 차트)
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

providers = df["provider"].unique()

for i, provider in enumerate(providers):
    ratings = df[df["provider"] == provider]["rating"]

    rating_counts = ratings.value_counts().sort_index()

    axes[i].pie(
        rating_counts,
        labels=rating_counts.index,
        autopct="%.1f%%",
        startangle=90,
        counterclock=False
    )
    axes[i].set_title(f"{provider} 평점 분포")

plt.tight_layout()
plt.show()

print("=" * 50)


# 3. 정규성 검정 (Shapiro-Wilk)
gcar_ratings = df[df["provider"] == "Gcar"]["rating"]
socar_ratings = df[df["provider"] == "Socar"]["rating"]

stat_g, p_g = stats.shapiro(gcar_ratings)
stat_s, p_s = stats.shapiro(socar_ratings)

print(f"Gcar 정규성 p-value : {p_g:.6f}")
print(f"Socar 정규성 p-value : {p_s:.6f}")

if p_g > 0.05 and p_s > 0.05:
    print("→ 두 집단 모두 정규성을 만족한다고 볼 수 있음.")
else:
    print("→ 정규성 불만족 → 비모수 검정(Mann-Whitney) 필요 가능성 높음.")

print("=" * 50)


# 4. 등분산성 검정 (Levene Test)
stat_levene, p_levene = stats.levene(gcar_ratings, socar_ratings)

print(f"등분산성 p-value : {p_levene:.6f}")

if p_levene > 0.05:
    print("→ 등분산성 만족.")
else:
    print("→ 등분산성 불만족.")

print("=" * 50)


# 5. 평균 차이 검정
normal = (p_g > 0.05) and (p_s > 0.05)

equal_var = (p_levene > 0.05)

print("=== 최종 검정 선택 ===")

if normal and equal_var:
    print("→ 독립표본 t-test 사용")
    stat_t, p_t = stats.ttest_ind(
        gcar_ratings, socar_ratings,
        equal_var=True
    )
    print(f"t-test p-value = {p_t:.10f}")

else:
    print("→ Mann-Whitney U 검정 사용 (비모수 검정)")
    stat_u, p_u = stats.mannwhitneyu(
        gcar_ratings, socar_ratings,
        alternative="two-sided"
    )
    print(f"Mann-Whitney U p-value = {p_u:.10f}")

# Gcar & Socar의 각 카테고리의 리뷰


In [ ]:
import seaborn as sns
import re

df["review_text"] = df["review_text"].astype(str)


# 1. 카테고리 필터 함수 정의
#    - 주어진 키워드를 포함하는 리뷰만 필터링
def filter_reviews_by_keywords(df, text_col, keywords):
    pattern = "|".join([fr"{re.escape(kw)}(\w)?" for kw in keywords])
    mask = df[text_col].str.contains(pattern, flags=re.IGNORECASE, na=False)
    return df[mask].copy()


# 2. 고객센터 관련 리뷰만 추출
cs_keywords = ["고객센터", "고객", "센터", "전화", "상담"]

df_cs = filter_reviews_by_keywords(df, "review_text", cs_keywords)
print(f"[INFO] 고객센터 관련 리뷰 수: {len(df_cs)}")


# 3. 브랜드별 평점 개수 & 비율 계산
rating_counts = (
    df_cs
    .groupby(["provider", "rating"])
    .size()
    .reset_index(name="count")
)

rating_counts["ratio"] = (
    rating_counts
    .groupby("provider")["count"]
    .transform(lambda x: x / x.sum() * 100)
)

print(rating_counts)


# 4. 시각화 (막대그래프)
plt.figure(figsize=(8, 5))

ax = sns.barplot(
    data=rating_counts,
    x="rating",
    y="ratio",
    hue="provider",
    palette="Set2"
)

plt.title("‘고객센터 관련 키워드’ 리뷰의 평점 비율 (브랜드별)")
plt.xlabel("평점")
plt.ylabel("비율(%)")
plt.legend(title="브랜드")


# 5. 막대 위에 비율 숫자 표시
for container in ax.containers:
    ax.bar_label(
        container,
        fmt="%.1f%%",
        label_type="edge",
        fontsize=9,
        padding=2
    )

plt.tight_layout()
plt.show()

# G car 워드클라우드

In [ ]:
import os, random
from PIL import Image, ImageFilter
from wordcloud import WordCloud


# 1. 파일 경로 & 옵션
logo_path = "/content/Miro Image (4).png"   # Gcar 로고 이미지
out_png   = "gcar_wordcloud_dense.png"

# 최종 이미지 해상도 (높을수록 선명 + 글자 더 촘촘)
width, height = 2000, 2000
bg_color  = "white"


# 2. 굵은 한글 폰트(Bold) 자동 탐색
CANDIDATE_FONTS = [
    "/System/Library/Fonts/AppleSDGothicNeoB.ttf",
    "/System/Library/Fonts/AppleSDGothicNeo.ttc",
    "/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf",
    "C:/Windows/Fonts/malgunbd.ttf",
    "C:/Windows/Fonts/malgun.ttf",
]
font_path = next((p for p in CANDIDATE_FONTS if os.path.exists(p)), None)
if not font_path:
    raise FileNotFoundError("Bold 한글 폰트를 찾을 수 없습니다. font_path를 직접 설정하세요.")


# 3. 색상 팔레트 (넷플릭스 스타일)
NETFLIX_PALETTE = ["#0B0B0B", "#B20710", "#E50914", "#F5F5F1"]

def netflix_color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    """
    워드클라우드 단어 색상 랜덤 선택 함수
    → 넷플릭스 느낌을 내기 위한 컬러 가중치
    """
    return random.choices(
        NETFLIX_PALETTE,
        weights=[0.38, 0.34, 0.24, 0.04],
        k=1
    )[0]


# 4. 로고 내부를 ‘검정=단어 채울 곳’으로 변환하는 마스크 함수
def make_mask_inside_black(path, target_size=(2000, 2000),
                           threshold=236, pad=30, thicken=3):
    """
    1) RGBA 로고 → 흰색 배경으로 통일
    2) 정사각형 캔버스에 중앙 배치
    3) target_size로 리사이즈
    4) threshold 기준으로 로고 내부만 검정(0), 외부는 흰색(255)
    5) MinFilter로 검정 영역 확장(로고 내부 면적 넓혀 촘촘함 강화)
    """
    img = Image.open(path).convert("RGBA")

    # 투명 배경 제거 → 흰 배경 합성
    bg = Image.new("RGBA", img.size, (255, 255, 255, 255))
    img = Image.alpha_composite(bg, img).convert("RGB")

    # 정사각형 캔버스에 로고 중앙 배치
    max_side = max(img.size)
    canvas = Image.new("RGB", (max_side + pad * 2, max_side + pad * 2), (255, 255, 255))
    offset = ((canvas.size[0] - img.size[0]) // 2,
              (canvas.size[1] - img.size[1]) // 2)
    canvas.paste(img, offset)

    # 원하는 사이즈로 확대 (LANCZOS: 고해상도 리사이즈)
    canvas = canvas.resize(target_size, Image.LANCZOS)

    # 그레이 변환 → 어두운 영역만 추출
    gray = canvas.convert("L").filter(ImageFilter.GaussianBlur(0.8))
    arr  = np.array(gray)

    # 로고 내부: 0(검정), 외부: 255(흰색)
    mask = np.where(arr < threshold, 0, 255).astype(np.uint8)

    # 로고 내부를 ‘팽창’시켜 단어가 더 촘촘하게 들어가도록 함
    if thicken and thicken >= 3 and thicken % 2 == 1:
        pil_mask = Image.fromarray(mask)
        for _ in range(2):  # 너무 많이 팽창되면 모양이 무너짐 → 적당히 1~2회
            pil_mask = pil_mask.filter(ImageFilter.MinFilter(size=thicken))
        mask = np.array(pil_mask)

    return mask


# ---- 마스크 생성 ----
mask = make_mask_inside_black(
    logo_path,
    target_size=(width, height),
    threshold=238,
    pad=40,
    thicken=5
)


# 5. 워드클라우드 생성
freqs = dict([
    ("안된다", 2300), ("고객", 1650), ("센터", 1200), ("아니다", 1050), ("앱", 990),
    ("시간", 940), ("반납", 920), ("전화", 880), ("예약", 860), ("인증", 750),
    ("쓰다", 720), ("해주다", 690), ("업데이트", 660), ("다시", 640), ("오류", 580),
    ("등록", 610), ("문의", 600), ("처리", 590), ("로그인", 500), ("문제", 510),
    ("가다", 560), ("연락", 540), ("빌리다", 520), ("확인", 480), ("사고", 470),
    ("되다", 460), ("가입", 450), ("쿠폰", 430), ("결제", 400), ("취소", 390),
    ("나오다", 380), ("오다", 370), ("상담", 350), ("모르다", 340), ("사람", 330),
    ("어떻다", 320), ("입력", 300), ("연결", 290), ("회원", 280), ("포인트", 270),
    ("생각", 260), ("해보다", 250)
])


wc = WordCloud(
    font_path=font_path,
    width=width, height=height,
    background_color=bg_color,
    mask=mask,
    mode="RGBA",

    # 워드클라우드 밀도 조절 핵심 파라미터
    scale=3,             # 해상도 증가 → 글자 선명
    max_words=1500,      # 단어 수 증가 → 내부 공간 더 촘촘하게
    min_font_size=10,    # 과도한 큰 공백 방지
    margin=0,            # 글자 간 여백 최소화
    repeat=True,         # 빈 공간 메우기 위해 단어 반복 허용
    relative_scaling=0.35,
    prefer_horizontal=0.9,
    collocations=False,  # 단어쌍 묶임 방지 (단어 다양성↑)
    random_state=42
)

wc.generate_from_frequencies(freqs)
wc.recolor(color_func=netflix_color_func)


# 6. 시각화 & 저장
plt.figure(figsize=(9, 9))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.tight_layout()
plt.show()

wc.to_file(out_png)
print("저장 완료:", os.path.abspath(out_png))

# 리뷰 주요 키워드 시각화
- 고객센터·전화 상위 키워드
- 앱·예약 상위 키워드
- 시간·반납 상위 키워드

In [ ]:
import re
from collections import Counter


# 1. 카테고리(그룹) 정의
GROUPS = {
    "고객센터·전화": ["고객센터", "전화"],
    "앱·예약":      ["앱", "어플", "예약"],
    "시간·반납":    ["시간", "반납"],
}


# 2. 그룹별 핵심 키워드 추출 함수
def extract_top_keywords(df: pd.DataFrame,
                         group_name: str,
                         seed_keywords: list,
                         top_n: int = 15,
                         min_len: int = 2,
                         min_count: int = 2,
                         show_plot: bool = True):
    pattern = "|".join(map(re.escape, seed_keywords))
    sub = df[df["clean_text"].str.contains(pattern, na=False)]
    bag = []
    seed_set = set(seed_keywords)

    for txt in sub["clean_text"]:
        for w in tokenize(txt):
            if len(w) < min_len:
                continue
            if w in STOPWORDS:
                continue
            if w in seed_set:
                continue
            bag.append(w)

    freq = Counter(bag)

    if min_count > 1:
        freq = Counter({k: v for k, v in freq.items() if v >= min_count})

    top = pd.Series(dict(freq.most_common(top_n)), dtype="int64")

    if show_plot and not top.empty:
        plt.figure(figsize=(6, 4))
        sorted_top = top.sort_values(ascending=True)

        ax = sorted_top.plot(kind="barh")
        plt.title(f"[{group_name}] 관련 리뷰 주요 키워드", fontsize=12)
        plt.xlabel("빈도")
        plt.ylabel("키워드")

        max_val = sorted_top.max()
        offset = max_val * 0.01 if max_val > 0 else 0.1
        for patch in ax.patches:
            width = patch.get_width()
            y = patch.get_y() + patch.get_height() / 2
            ax.text(
                width + offset,
                y,
                f"{int(width)}",
                va="center",
                ha="left",
                fontsize=9
            )

        plt.tight_layout()
        plt.show()
    return top, sub


# 3. 그룹별 실행 (고객센터·전화 / 앱·예약 / 시간·반납)
# 전처리: 부정 리뷰만 사용 + clean_text 생성

TOP_N     = 15
MIN_LEN   = 2
MIN_COUNT = 2

top_call, sub_call = extract_top_keywords(
    neg_df,
    group_name="고객센터·전화",
    seed_keywords=GROUPS["고객센터·전화"],
    top_n=TOP_N,
    min_len=MIN_LEN,
    min_count=MIN_COUNT
)

top_app, sub_app = extract_top_keywords(
    neg_df,
    group_name="앱·예약",
    seed_keywords=GROUPS["앱·예약"],
    top_n=TOP_N,
    min_len=MIN_LEN,
    min_count=MIN_COUNT
)

top_time, sub_time = extract_top_keywords(
    neg_df,
    group_name="시간·반납",
    seed_keywords=GROUPS["시간·반납"],
    top_n=TOP_N,
    min_len=MIN_LEN,
    min_count=MIN_COUNT
)


# 4. 상위 키워드 표로 확인 + 예문 출력 함수
print("\n[고객센터·전화] 상위 키워드")
print(top_call)

print("\n[앱·예약] 상위 키워드")
print(top_app)

print("\n[시간·반납] 상위 키워드")
print(top_time)


def show_examples(sub_df: pd.DataFrame, n: int = 5):
    """
    특정 그룹(sub_df)에서 예시 리뷰 n개 출력
    """
    print(sub_df["review_text"].dropna().head(n).to_string(index=False))


print("\n[고객센터·전화] 예문 샘플:")
show_examples(sub_call, n=5)

print("\n[앱·예약] 예문 샘플:")
show_examples(sub_app, n=5)

print("\n[시간·반납] 예문 샘플:")
show_examples(sub_time, n=5)


# 부정 리뷰 통계

- 카테고리별 부정 비율이 2021→2025 동안 '감소 추세'인지 통계적으로 검정
- 두 가지 방법 사용:
    1) 선형 추세 검정: OLS + Newey-West(HAC) 표준오차
       - 귀무가설: 기울기 β1 = 0 (추세 없음)
       - 대립가설: 기울기 β1 < 0 (감소 추세)
    2) Mann–Kendall 비모수 검정 (단조 감소 여부)
- 다중검정(Benjamini–Hochberg FDR) 보정 적용

In [ ]:
import statsmodels.api as sm
from scipy.stats import t as t_dist

# 0) 전제 조건
#    → 메인 분석 스크립트에서 아래 변수가 이미 정의되어 있어야
#       - ratio_A_gcar : 연도별 지표 A (전체 대비 해당 카테고리 '부정' 비율)
#       - ratio_B_gcar : 연도별 지표 B (부정 리뷰 내 해당 카테고리 비중)
#       - CAT_PATTERNS : 카테고리 이름 dict (키: 카테고리명)
#       - OUT_DIR      : 결과 저장 폴더 경로
#       - YEAR_MIN_SEL, YEAR_MAX_SEL : 분석할 연도 구간 (예: 2021~2025)

req_vars = ["ratio_A_gcar", "ratio_B_gcar",
            "CAT_PATTERNS", "OUT_DIR",
            "YEAR_MIN_SEL", "YEAR_MAX_SEL"]

missing = [v for v in req_vars if v not in globals()]
if missing:
    raise RuntimeError(
        f"다음 변수가 필요합니다: {missing}  → 먼저 메인 스크립트를 실행하세요."
    )



# 1) 선형 추세 검정 (OLS + Newey-West(HAC) 표준오차)
#    - y: index=연도, 값=카테고리 비율
#    - β1 < 0 인지를 보는 단측 검정 (감소 추세?)

def ols_trend_test(y: pd.Series) -> dict:
    """
    y : 연도별 지표 시계열 (index=연도, values=비율)
    반환 dict:
        - beta1    : 기울기 추정치
        - t        : t-통계량
        - p_two    : 양측 p-value (β1 != 0)
        - p_one    : 단측 p-value (β1 < 0, 감소 가설)
        - n        : 사용된 관측치 개수
        - df_resid : 잔차 자유도
    """
    y = y.dropna()
    if len(y) < 3:
        return {
            "beta1": np.nan, "t": np.nan,
            "p_two": np.nan, "p_one": np.nan,
            "n": len(y), "df_resid": np.nan
        }

    # 시간 축: 0, 1, 2, ... (연도 간격을 1로 둔 단순 직선 회귀)
    t = np.arange(len(y), dtype=float)
    X = sm.add_constant(t)  # 상수항 + 기울기

    model = sm.OLS(y.values, X)

    # Newey-West 래그 수: sqrt(T) 정도로 보수적으로 설정
    T = len(y)
    maxlags = max(1, int(np.sqrt(T)))

    # HAC(Newey-West) 공분산 사용
    res = model.fit(cov_type="HAC", cov_kwds={"maxlags": maxlags})

    beta1 = res.params[1]
    tval  = res.tvalues[1]
    dfree = res.df_resid

    # 양측 p-value
    p_two = res.pvalues[1]
    # 단측 p-value (감소 가설: β1 < 0 → 왼쪽 꼬리)
    p_one = t_dist.cdf(tval, df=dfree)

    return {
        "beta1": float(beta1),
        "t": float(tval),
        "p_two": float(p_two),
        "p_one": float(p_one),
        "n": int(T),
        "df_resid": float(dfree),
    }



# 2) Mann–Kendall 단조 감소 검정 (비모수)
#    - 시계열이 '단조 감소' 방향인지 보는 검정

def mann_kendall_decreasing(y: pd.Series) -> dict:
    """
    y : 연도별 지표 시계열 (Series 또는 array-like)
    반환 dict:
        - S      : Mann–Kendall S 통계량
        - varS   : S의 분산
        - z      : 표준화 z 통계량
        - p_one  : 단측 p-value (감소 가설: z < 0 일수록 유의)
        - n      : 사용된 관측치 개수
    """
    from scipy.stats import norm

    y = y.dropna().values
    n = len(y)
    if n < 3:
        return {
            "S": np.nan, "varS": np.nan,
            "z": np.nan, "p_one": np.nan,
            "n": n
        }

    # S 통계량 계산
    S = 0
    for i in range(n - 1):
        diff = y[i+1:] - y[i]
        S += np.sum(np.sign(diff))

    # 분산 (동점 없는 기본 공식 사용)
    varS = (n * (n - 1) * (2 * n + 5)) / 18.0

    # z 값 계산 (S>0, S<0 에 따라 보정)
    if S > 0:
        z = (S - 1) / np.sqrt(varS)
    elif S < 0:
        z = (S + 1) / np.sqrt(varS)
    else:
        z = 0.0

    # 단측 p-value (감소 가설: z가 작을수록 유의)
    p_one = norm.cdf(z)

    return {
        "S": float(S),
        "varS": float(varS),
        "z": float(z),
        "p_one": float(p_one),
        "n": int(n),
    }



# 3) 카테고리별로 A/B 지표에 대해 추세 검정 실행
#    - A: 전체 리뷰 중 '해당 카테고리 & 부정' 비율
#    - B: 부정 리뷰 내부에서 해당 카테고리 비중

def run_all_tests(ratio_df: pd.DataFrame, label: str) -> pd.DataFrame:
    """
    ratio_df : index=연도, columns=카테고리 이름 (비율 값)
    label    : "A" 또는 "B" (어떤 지표인지 표시용)

    반환:
        각 카테고리별로
          - 선형추세(OLS+HAC) 결과
          - Mann–Kendall 결과
          - FDR 보정 p값
        를 담은 DataFrame
    """
    rows = []

    # 정의된 모든 카테고리에 대해 반복
    for cat in CAT_PATTERNS.keys():
        if cat not in ratio_df.columns:
            continue

        # 선택 연도 구간(YEAR_MIN_SEL~YEAR_MAX_SEL)만 사용
        y = ratio_df[cat].copy()
        y = y.reindex(range(YEAR_MIN_SEL, YEAR_MAX_SEL + 1))

        # 1) 선형 추세 검정
        lin = ols_trend_test(y)

        # 2) Mann–Kendall 단조 감소 검정
        mk = mann_kendall_decreasing(y)

        rows.append({
            "metric": label,        # A or B
            "category": cat,        # 카테고리 이름
            "n_points": lin["n"],   # 사용된 연도 개수
            "beta1_slope": lin["beta1"],
            "t_stat": lin["t"],
            "p_one_linear": lin["p_one"],   # 단측 p (감소 가설)
            "p_two_linear": lin["p_two"],   # 양측 p
            "mk_S": mk["S"],
            "mk_z": mk["z"],
            "p_one_mk": mk["p_one"],        # 단측 p (감소 가설)
        })

    out = pd.DataFrame(rows)


    # 3-1) Benjamini–Hochberg FDR 보정
    #      - 단측 p값(p_one_linear, p_one_mk)에 대해 q-value 계산

    def bh_fdr(pvals: pd.Series) -> pd.Series:
        """
        Benjamini–Hochberg FDR 보정
        - pvals: 원본 p-value Series
        - 반환: 각 항목에 대한 q-value (FDR 보정 후 p값)
        """
        p = pvals.values.astype(float)
        m = np.sum(~np.isnan(p))  # 유효 p값 개수

        # NaN은 무한대로 취급해서 뒤로 밀기
        order = np.argsort(np.where(np.isnan(p), np.inf, p))

        ranked = np.empty_like(p, dtype=float)
        ranked[:] = np.nan

        prev = 0.0
        for i, idx in enumerate(order, start=1):
            if np.isnan(p[idx]):
                continue
            # BH: q_i = p_i * (m / rank_i)
            val = p[idx] * m / i
            prev = max(prev, val)      # monotonic 증가 보장
            ranked[idx] = min(prev, 1.0)

        return pd.Series(ranked, index=pvals.index)

    # 선형 추세 / Mann–Kendall 각각에 대해 FDR 보정 q값 추가
    out["q_one_linear"] = bh_fdr(out["p_one_linear"])
    out["q_one_mk"]     = bh_fdr(out["p_one_mk"])

    # q<0.05 인 경우만 "✓" 표시해서 해석하기 쉽게
    out["sig_linear(q<0.05)"] = (out["q_one_linear"] < 0.05).map({True: "✓", False: ""})
    out["sig_mk(q<0.05)"]     = (out["q_one_mk"] < 0.05).map({True: "✓", False: ""})

    # 보기 좋게 정렬
    out = out.sort_values(["metric", "category"]).reset_index(drop=True)
    return out


# A/B 지표 각각에 대해 검정 실행
res_A = run_all_tests(ratio_A_gcar, "A")  # 전체 대비 '해당 카테고리 부정' 비율
res_B = run_all_tests(ratio_B_gcar, "B")  # 부정 리뷰 내부 카테고리 구성비
res_all = pd.concat([res_A, res_B], axis=0).reset_index(drop=True)

# 요약 출력
print("\n=== 추세 검정 요약 (단측: '감소' 가설 기준) ===")
display_cols = [
    "metric", "category", "n_points",
    "beta1_slope", "t_stat",
    "p_one_linear", "q_one_linear", "sig_linear(q<0.05)",
    "mk_S", "mk_z", "p_one_mk", "q_one_mk", "sig_mk(q<0.05)"
]
print(res_all[display_cols].round(4))

# CSV 저장
csv_path = os.path.join(OUT_DIR, "Gcar_2021_2025_trend_tests.csv")
res_all.to_csv(csv_path, index=False, encoding="utf-8-sig")
print(f"\n📄 검정 결과 저장: {csv_path}")



# 4) 저장된 그래프 PNG 파일 검증

def verify_saved_images(out_dir: str, required_keywords=None):
    """
    out_dir 내 PNG 파일 리스트업 + 파일 크기 리포트

    required_keywords:
        - 예: ['부정', '구성비', 'changes_A']
        - 지정하면, 파일명에 이 키워드 중 하나라도 포함되는지 ok 여부를 함께 표시
    """
    from glob import glob

    pngs = sorted(glob(os.path.join(out_dir, "*.png")))
    print(f"\n[이미지 검증] 폴더: {out_dir}")

    if not pngs:
        print("저장된 PNG가 없습니다.")
        return pd.DataFrame(columns=["file", "size_bytes", "ok_keyword"])

    rows = []
    for p in pngs:
        size = os.path.getsize(p)
        ok_kw = True
        if required_keywords:
            ok_kw = any(k in os.path.basename(p) for k in required_keywords)

        rows.append({
            "file": os.path.basename(p),
            "size_bytes": size,
            "ok_keyword": ok_kw,
        })

    dfv = pd.DataFrame(rows).sort_values("file")
    print(dfv.to_string(index=False))
    return dfv

# 부정 리뷰 통계 검정 및 시각화
- 카테고리
  - 고객센터
  - 앱/어플
  - 반납/예약
  - 벌금/과금/패널티
- 연도별 추이 그래프
- 분기별 추이 그래프
- 모든 카테고리의 분기별 꼭지점 그래프
- 개선율 계산

In [ ]:
CSV_PATH = "/content/reviews_sentiment_all.csv"
OUT_DIR = "/content/outputs_gcar"
os.makedirs(OUT_DIR, exist_ok=True)

BRAND_TARGET = "Gcar"
YEAR_MIN_SEL, YEAR_MAX_SEL = 2021, 2025


# 1. 데이터 로드
df = pd.read_csv(CSV_PATH)

# 기본 컬럼 이름
text_col = "review_text"
date_col = "review_date"
brand_col = "provider"
sent_col = "sentiment_label"
rating_col = "rating"

# 날짜 처리
df["year"] = pd.to_datetime(df[date_col], errors="coerce").dt.year
df["quarter"] = pd.to_datetime(df[date_col], errors="coerce").dt.quarter

# Gcar만 사용
df = df[df[brand_col].str.contains("gcar", case=False, na=False)].copy()

# 연도 범위 필터
df = df[df["year"].between(YEAR_MIN_SEL, YEAR_MAX_SEL)]

# 텍스트 소문자화
df[text_col] = df[text_col].astype(str).str.lower()


# 2. 부정 리뷰 플래그
NEG = {"negative", "부정", "bad", "-1"}

def is_negative(row):
    if row[sent_col] in NEG:
        return True
    try:
        return float(row[rating_col]) <= 2
    except:
        return False

df["is_negative"] = df.apply(is_negative, axis=1)


# 3. 카테고리 패턴
CAT_PATTERNS = {
    "고객센터/전화": re.compile(r"(고객센터|상담|전화|문의|콜센터)"),
    "앱/어플": re.compile(r"(앱|어플|로그인|오류|버그|에러|인증|업데이트)"),
    "반납/예약": re.compile(r"(반납|예약|취소|연장|확정|종료|시작)"),
    "벌금/과금/패널티": re.compile(r"(벌금|과금|패널티|위약금|연체료)")
}

def match_cat(pattern, text):
    return bool(pattern.search(text))

# 카테고리 컬럼 생성
for cat, pat in CAT_PATTERNS.items():
    df[cat] = df[text_col].apply(lambda t: match_cat(pat, t))


# 4. 지표 A / B 계산
#    A: 전체 리뷰 중 해당 카테고리 부정 비율
#    B: 부정 리뷰 내부에서 카테고리 구성비

# 카테고리별 부정 여부
neg_cols = []
for cat in CAT_PATTERNS.keys():
    col = f"neg__{cat}"
    df[col] = df["is_negative"] & df[cat]
    neg_cols.append(col)

# ---- 연도 기준 ----
grp_all = df.groupby("year")
A_num = grp_all[neg_cols].sum()
A_num.columns = list(CAT_PATTERNS.keys())

A_den = grp_all.size().rename("n_all")
ratio_A = (A_num.div(A_den, axis=0)).fillna(0)

# 부정 리뷰 내부
grp_neg = df[df["is_negative"]].groupby("year")
B_num = grp_neg[list(CAT_PATTERNS.keys())].sum()
B_den = grp_neg.size().rename("n_neg")
ratio_B = (B_num.div(B_den, axis=0)).fillna(0)

# ---- 분기 기준 ----
grp_all_q = df.groupby(["year", "quarter"])
A_num_q = grp_all_q[neg_cols].sum()
A_num_q.columns = list(CAT_PATTERNS.keys())

A_den_q = grp_all_q.size().rename("n_all_q")
ratio_A_q = (A_num_q.div(A_den_q, axis=0)).fillna(0)

grp_neg_q = df[df["is_negative"]].groupby(["year", "quarter"])
B_num_q = grp_neg_q[list(CAT_PATTERNS.keys())].sum()
B_den_q = grp_neg_q.size().rename("n_neg_q")
ratio_B_q = (B_num_q.div(B_den_q, axis=0)).fillna(0)


# 5. 그래프 함수
def annotate_points(xs, ys):
    ax = plt.gca()
    for x, y in zip(xs, ys):
        ax.text(x, y, f"{y*100:.1f}%", ha="center", va="bottom", fontsize=9)

def plot_year(ratio_sub, title, ylabel):
    cats = list(CAT_PATTERNS.keys())
    ys = ratio_sub.index.values
    for cat in cats:
        vals = ratio_sub[cat].values
        plt.figure(figsize=(8,5))
        plt.plot(ys, vals, marker="o")
        annotate_points(ys, vals)
        plt.title(f"{cat} · {title}")
        plt.xlabel("연도"); plt.ylabel(ylabel + " (%)")
        plt.grid(alpha=0.3)
        plt.tight_layout()
        fname = f"{cat}_{title}_year.png"
        plt.savefig(os.path.join(OUT_DIR, fname))
        plt.show()

def plot_quarter(ratio_sub, title, ylabel):
    cats = list(CAT_PATTERNS.keys())
    idx = ratio_sub.index
    xs = np.arange(len(idx))
    labels = [f"{y}Q{q}" for (y, q) in idx]

    for cat in cats:
        vals = ratio_sub[cat].values
        plt.figure(figsize=(10,5))
        plt.plot(xs, vals, marker="o")
        annotate_points(xs, vals)
        plt.xticks(xs, labels, rotation=45)
        plt.title(f"{cat} · {title}")
        plt.xlabel("분기"); plt.ylabel(ylabel + " (%)")
        plt.grid(alpha=0.3)
        plt.tight_layout()
        fname = f"{cat}_{title}_quarter.png"
        plt.savefig(os.path.join(OUT_DIR, fname))
        plt.show()


# 6. 그래프 생성
plot_year(ratio_A, "전체 대비 부정 비율", "비율")
plot_year(ratio_B, "부정 내부 구성비", "구성비")

plot_quarter(ratio_A_q, "전체 대비 부정 비율 — 분기", "비율")
plot_quarter(ratio_B_q, "부정 내부 구성비 — 분기", "구성비")


# 7. 개선율 계산 (2021→2025)
def safe_change(v1, v2):
    return ((v2 - v1) / v1.replace(0, np.nan) * 100).replace([np.inf, -np.inf], np.nan).fillna(0).round(1)

chg = safe_change(ratio_A.loc[YEAR_MIN_SEL], ratio_A.loc[YEAR_MAX_SEL])
chg_df = chg.to_frame(name="change_percent(2021→2025)")
chg_df.to_csv(os.path.join(OUT_DIR, "Gcar_changes_A_2021_2025.csv"), encoding="utf-8-sig")

plt.figure(figsize=(9,5))
plt.bar(chg.index, chg.values)
for i, v in enumerate(chg.values):
    plt.text(i, v, f"{v:+.1f}%", ha="center", va="bottom")
plt.title("Gcar 카테고리 개선율 (2021→2025)")
plt.xticks(rotation=15)
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "Gcar_changes_A_2021_2025.png"))
plt.show()

print("완료: Gcar 부정 리뷰 통계 + 4개 카테고리 분석 완료!")